In [1]:
import os
import json
import statistics
import pandas as pd
from collections import Counter

In [2]:
def filter_zero_columns(dataframe): #delete columnds that are 0.0 down the entire line
    for column in dataframe:
        if sum(dataframe[column]) == 0.0:
            del dataframe[column]

def zero_intervals(l):
    result = list()
    z = 0
    for i in range(1,len(l)):
        if l[i]-l[i-1] == 0:
            z += 1
        else:
            result.append(z)
            z = 0
    if z:
        result.append(z)
    return dict(Counter(result))

def diff_list(l):
    return [ float(float(l[i]) - float(l[i-1])) for i in range(1,len(l))]

def avg_nonzero_energy_increase(energy):
    return statistics.mean([ n for n in diff_list(energy) if n != 0])
def stdev_nonzero_energy_increase(energy):
    return statistics.stdev([ n for n in diff_list(energy) if n != 0 ])

In [3]:
results_dir = 'jolteon-results-subset'
os.chdir(results_dir)

In [4]:
datafiles = os.listdir()
datafilenames = list(set([ x.split('.')[0] for x in datafiles]))
datafilenames

['luindex_8_java',
 'eclipse_6_c-linklist',
 'batik_3_java',
 'avrora_7_c-dynamicarray',
 'h2o_4_java',
 'graphchi_4_c-linklist',
 'sunflow_6_c-linklist']

In [5]:
parts = datafilenames[0].split('_')
benchmark = parts[0]
iter = parts[1]
type = parts[2]

filename_base = datafilenames[0]
with open(f'{filename_base}.metadata.json') as fh: metadata = json.loads(fh.read())
data = pd.read_csv(f'{filename_base}.csv')
filter_zero_columns(data)

data

,socket,dram,core,pkg,timestamp
0,1,20228.5568,0.0,137037.5273,1615064809698108
1,2,15712.8222,0.0,188919.5948,1615064809698108
2,1,20228.5568,0.0,137037.5273,1615064809698146
3,2,15712.8222,0.0,188919.5948,1615064809698146
4,1,20228.5568,0.0,137037.5273,1615064809698181
...,...,...,...,...,...
129595,2,15718.4911,0.0,188956.9125,1615064811095735
129596,1,20228.6205,0.0,137059.1218,1615064811095756
129597,2,15718.4911,0.0,188956.9125,1615064811095756
129598,1,20228.6205,0.0,137059.1218,1615064811095776


In [9]:

result = {}
result['metadata'] = metadata

num_sockets = max(data['socket'])
for i in range(num_sockets): # filling out the socket level of result{}
    socket = i+1
    current = data[data.socket.eq(socket)].to_dict()
    del current['socket']
    result[socket] = current
    timestamps = result[socket]['timestamp']
    print(timestamps[:10])
    
for socket in result: # filling out the actual computations in result{}
    timestamps = result[socket]['timestamp']
    del result[socket]['timestamp']
    power_domains = list(result[socket])
    for powd in power_domains:
        energy = result[socket][powd]
        energy = [ energy[i] for i in energy ] #convert from dict<int,double> to list<double>
        result[socket][powd] = dict()
        result[socket][powd]['zero-intervals'] = zero_intervals(energy)
        result[socket][powd]['nonzero-energy-increase'] = dict()
        result[socket][powd]['nonzero-energy-increase']['avg'] = avg_nonzero_energy_increase(energy)
        result[socket][powd]['nonzero-energy-increase']['stdev'] = stdev_nonzero_energy_increase(energy)

result

TypeError: unhashable type: 'slice'

In [ ]:
with open(f'{filename_base}.stats.json','w') as fh: fh.write(json.dumps(result))